In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
import numpy as np
import pandas as pd
import scipy

# Instansiate the Plotly charting library.
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
# We use plotly.offline as this allows us to create interactive 
# visualisations without the use of an internet connection, 
# making our notebook more distributable to others. 
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

# The Cufflinks library allows us to directly bind 
# Pandas dataframes to Plotly charts. 
import cufflinks as cf
# Once again we use the Cufflinks library in offline mode. 
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options. We use these to make our interactive 
# visualisations more aesthetically appealing. 
from IPython.core.display import HTML
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from ipywidgets import interact, interact_manual, widgets

PROJ: proj_create_from_database: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name


In [3]:
df = pd.read_csv('data/freeman_well_4_eng.csv')

In [4]:
df.describe()

,Depth,GR,Log_ILD,DT,RHOB,NPHI,PHI,PERM,Facies,velocity,Facies_code
count,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7.286000e+03,7286.000000,7.286000e+03,7286.000000
mean,9517.276352,108.163436,16.647403,117.850574,2.330201,0.423016,0.206324,9.691201e+03,2.820615,9.447880e+05,1.820615
std,1056.427949,22.303899,167.046140,11.625088,0.081099,0.065935,0.052322,2.375493e+05,0.564634,4.050703e+05,0.564634
min,7682.500000,37.810800,0.081200,84.811200,1.863100,0.099500,0.055100,1.076985e-02,1.000000,5.128205e+02,0.000000
25%,8608.125000,101.914675,0.877900,108.402500,2.282625,0.384900,0.166700,1.836431e+00,3.000000,7.322923e+05,2.000000
50%,9518.750000,112.481050,1.025900,114.944650,2.347100,0.422100,0.195400,6.885164e+00,3.000000,9.747539e+05,2.000000
75%,10430.875000,120.165375,1.365575,127.965500,2.391600,0.461475,0.237000,4.675485e+01,3.000000,1.139082e+06,2.000000
max,11357.000000,178.320800,1950.000000,158.538400,2.564600,0.656900,0.507700,1.211313e+07,3.000000,1.231527e+07,2.000000


In [5]:
df.isnull().sum()

Depth          0
GR             0
Log_ILD        0
DT             0
RHOB           0
NPHI           0
PHI            0
PERM           0
Facies         0
velocity       0
Facies_code    0
dtype: int64

In [6]:
df.head()

,Depth,GR,Log_ILD,DT,RHOB,NPHI,PHI,PERM,Facies,velocity,Facies_code
0,7682.5,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
1,7683.0,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
2,7683.5,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
3,7684.0,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0
4,7684.5,39.0321,0.9332,137.507,2.2382,0.5983,0.2657,175.29364,1.0,1.071582e+06,0


In [7]:
df.describe()

,Depth,GR,Log_ILD,DT,RHOB,NPHI,PHI,PERM,Facies,velocity,Facies_code
count,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7.286000e+03,7286.000000,7.286000e+03,7286.000000
mean,9517.276352,108.163436,16.647403,117.850574,2.330201,0.423016,0.206324,9.691201e+03,2.820615,9.447880e+05,1.820615
std,1056.427949,22.303899,167.046140,11.625088,0.081099,0.065935,0.052322,2.375493e+05,0.564634,4.050703e+05,0.564634
min,7682.500000,37.810800,0.081200,84.811200,1.863100,0.099500,0.055100,1.076985e-02,1.000000,5.128205e+02,0.000000
25%,8608.125000,101.914675,0.877900,108.402500,2.282625,0.384900,0.166700,1.836431e+00,3.000000,7.322923e+05,2.000000
50%,9518.750000,112.481050,1.025900,114.944650,2.347100,0.422100,0.195400,6.885164e+00,3.000000,9.747539e+05,2.000000
75%,10430.875000,120.165375,1.365575,127.965500,2.391600,0.461475,0.237000,4.675485e+01,3.000000,1.139082e+06,2.000000
max,11357.000000,178.320800,1950.000000,158.538400,2.564600,0.656900,0.507700,1.211313e+07,3.000000,1.231527e+07,2.000000


In [8]:
from sklearn import preprocessing
from sklearn import utils
lab_enc = preprocessing.LabelEncoder()

In [9]:
df['Facies'].count()

7286

In [10]:
df['Facies']= lab_enc.fit_transform(df['Facies'])

In [11]:
df['Facies'].unique()

array([0, 1, 2], dtype=int64)

In [12]:
@interact
def correlations(column1=list(df.select_dtypes('number').columns), 
                 column2=list(df.select_dtypes('number').columns)):
    print(f"Correlation: {df[column1].corr(df[column2])}")

interactive(children=(Dropdown(description='column1', options=('Depth', 'GR', 'Log_ILD', 'DT', 'RHOB', 'NPHI',…

In [13]:
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:]):
    if x == y:
        print(f"Please select seperate variables for X and Y")
    else:
        df.iplot(kind='scatter', x=x, y=y, mode='markers', 
                 xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}')
        ## if you are using Google Colab, comment out the above line of code and uncomment the lines below
        #fig = px.scatter(df, x=x, y=y, title=f'{y.title()} vs {x.title()}')
        #fig.show(renderer="colab")

interactive(children=(Dropdown(description='x', options=('Depth', 'GR', 'Log_ILD', 'DT', 'RHOB', 'NPHI', 'PHI'…

In [14]:
cscales = ['Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Bluered', 'RdBu',
            'Reds', 'Blues', 'Picnic', 'Rainbow', 'Portland', 'Jet',
            'Hot', 'Blackbody', 'Earth', 'Electric', 'Viridis', 'Cividis']

# We use the Figure Factory module of Plotly, which
# defines many unique and powerful plots to be used
# in Python. 
# For more info, see: https://plot.ly/python/figure-factory-subplots/
import plotly.figure_factory as ff

corrs = df.corr()

@interact
def plot_corrs(colorscale=cscales):
    figure = ff.create_annotated_heatmap(z = corrs.round(2).values, 
                                     x =list(corrs.columns), 
                                     y=list(corrs.index), 
                                     colorscale=colorscale,
                                     annotation_text=corrs.round(2).values)
    iplot(figure)
    ## if you are using Google Colab, comment out the above line of code and uncomment the line below
    #figure.show(renderer="colab")

interactive(children=(Dropdown(description='colorscale', options=('Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Blue…

In [15]:
@interact_manual
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cf.themes.THEMES.keys()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    if x == y:
        print(f"Please select seperate variables for X and Y")
    else:
        df.iplot(kind='scatter', x=x, y=y, mode='markers', 
                 xTitle=x.title(), yTitle=y.title(), 
                 text='Depth',
                 title=f'{y.title()} vs {x.title()}',
                theme=theme, colorscale=colorscale)
        ## if you are using Google Colab, comment out the above line of code and uncomment the line below
        #fig = px.scatter(df, x=x, y=y, title=f'{y.title()} vs {x.title()}')
        #fig.show(renderer="colab")

interactive(children=(Dropdown(description='x', options=('Depth', 'GR', 'Log_ILD', 'DT', 'RHOB', 'NPHI', 'PHI'…

In [17]:
final = df.drop(['NPHI', 'DT'], axis = 'columns')
final

,Depth,GR,Log_ILD,RHOB,PHI,PERM,Facies,velocity,Facies_code
0,7682.5,39.0321,0.9332,2.2382,0.2657,175.29364,0,1.071582e+06,0
1,7683.0,39.0321,0.9332,2.2382,0.2657,175.29364,0,1.071582e+06,0
2,7683.5,39.0321,0.9332,2.2382,0.2657,175.29364,0,1.071582e+06,0
3,7684.0,39.0321,0.9332,2.2382,0.2657,175.29364,0,1.071582e+06,0
4,7684.5,39.0321,0.9332,2.2382,0.2657,175.29364,0,1.071582e+06,0
...,...,...,...,...,...,...,...,...,...
7281,11355.0,162.1151,0.9417,2.3986,0.1622,1.49274,2,1.061909e+06,2
7282,11355.5,162.1151,0.9418,2.3986,0.1622,1.49274,2,1.061797e+06,2
7283,11356.0,162.1151,0.9418,2.3986,0.1622,1.49274,2,1.061797e+06,2
7284,11356.5,162.1151,0.9419,2.3986,0.1622,1.49274,2,1.061684e+06,2


In [18]:
final.describe()

,Depth,GR,Log_ILD,RHOB,PHI,PERM,Facies,velocity,Facies_code
count,7286.000000,7286.000000,7286.000000,7286.000000,7286.000000,7.286000e+03,7286.000000,7.286000e+03,7286.000000
mean,9517.276352,108.163436,16.647403,2.330201,0.206324,9.691201e+03,1.820615,9.447880e+05,1.820615
std,1056.427949,22.303899,167.046140,0.081099,0.052322,2.375493e+05,0.564634,4.050703e+05,0.564634
min,7682.500000,37.810800,0.081200,1.863100,0.055100,1.076985e-02,0.000000,5.128205e+02,0.000000
25%,8608.125000,101.914675,0.877900,2.282625,0.166700,1.836431e+00,2.000000,7.322923e+05,2.000000
50%,9518.750000,112.481050,1.025900,2.347100,0.195400,6.885164e+00,2.000000,9.747539e+05,2.000000
75%,10430.875000,120.165375,1.365575,2.391600,0.237000,4.675485e+01,2.000000,1.139082e+06,2.000000
max,11357.000000,178.320800,1950.000000,2.564600,0.507700,1.211313e+07,2.000000,1.231527e+07,2.000000


In [19]:
final.isnull().sum()

Depth          0
GR             0
Log_ILD        0
RHOB           0
PHI            0
PERM           0
Facies         0
velocity       0
Facies_code    0
dtype: int64

In [20]:
final.dtypes

Depth          float64
GR             float64
Log_ILD        float64
RHOB           float64
PHI            float64
PERM           float64
Facies           int64
velocity       float64
Facies_code      int64
dtype: object

In [21]:
X = final.drop('PERM',axis='columns')
y = final.PERM

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X, y)

reg.coef_

LinearRegression()

array([ 3.48343703e+01,  1.30956469e+03,  2.12466121e+01, -3.42163370e+07,
       -5.13579168e+07, -8.84813339e+03, -7.99167622e-03, -8.84813336e+03])

In [22]:
from sklearn.ensemble import RandomForestRegressor

# Load well log data

# Define input features and target variable
X = final[['Log_ILD', 'RHOB', 'velocity', 'GR', 'Facies', 'DEPT']]
y = final['PERM']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest regressor model
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# Make predictions on test data
y_pred = rf.predict(X_test)

# Calculate model performance metrics
mae = np.mean(abs(y_test - y_pred))
mse = np.mean((y_test - y_pred) ** 2)
rmse = np.sqrt(mse)
r2 = rf.score(X_test, y_test)

# Make a prediction for new well log data
new_data = pd.DataFrame({'RES_DEP': [2.5], 'DEN': [2.3], 'velocity': [3.5], 'GR': [40], 'Facies': [1], 'DEPT': [2000]})
new_pp = rf.predict(new_data)

print(new_pp)

KeyError: "['RES_DEP', 'DEN', 'DEPT'] not in index"

In [ ]:
print(r2)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define dictionary of models and their hyperparameters
model_params = {
    'svm': {
        'model': svm.SVR(),
        'params' : {
            'C': [0.1, 1, 10],
            'epsilon': [0.01, 0.1, 1],
            'kernel': ['linear', 'rbf', 'poly']
        }  
    },
    'random_forest': {
        'model': RandomForestRegressor(),
        'params' : {
            'n_estimators': [1, 5, 10],
            'max_depth': [5, 10, 15]
        }
    },
    'decision_tree': {
        'model': DecisionTreeRegressor(),
        'params': {
            'max_depth': [5, 10, 15]
        }
    },
    'neural_network': {
        'model': MLPRegressor(),
        'params': {
            'hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'activation': ['relu', 'tanh'],
            'alpha': [0.0001, 0.001, 0.01]
        }
    }
}

In [ ]:
# Loop over each model and perform grid search with cross-validation to find best hyperparameters
for model_name, mp in model_params.items():
    print('Training', model_name)
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
    clf.fit(X_train, y_train)
    print('Best hyperparameters:', clf.best_params_)
    print('Best cross-validation score:', np.sqrt(-clf.best_score_))
    
    # Evaluate best model on test set
    y_pred = clf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = clf.score(X_test, y_test)
    print('Test set performance - RMSE:', rmse, ', R-squared:', r2)


save model

In [ ]:
import pickle

In [ ]:
with open('model_pickle', 'wb') as file:
    pickle.dump(model,file)

load model

In [ ]:
with open('model_pickle', 'rb') as file:
    mp = pickle.load(file)